In [32]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.mstats import winsorize
from numpy import array
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore', category=UserWarning)
pd.options.display.float_format = '{:.2f}'.format

In [33]:
# Mathieu
path = r'C:\Users\mperan\OneDrive - IESEG\Desktop\DP Analysis\Group Project\Project data\\'

In [34]:
campaigns = pd.read_pickle('campaigns_cleaned.pkl')
gifts = pd.read_pickle('gifts_cleaned.pkl')

In [35]:
campaigns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   campaignID   367 non-null    object        
 1   date         367 non-null    datetime64[ns]
 2   lettersSent  367 non-null    int32         
 3   CostUnit     367 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int32(1), object(1)
memory usage: 10.2+ KB


In [36]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# Define the time window for each dataset
time_window = relativedelta(years=5)

# Drop dates of campaigns
drop_score = pd.to_datetime('2020-02-01')

# Define the gap period: 2 weeks
gap_period = timedelta(days=14)

# Calculate the cutoff dates
cutoff_date_score = drop_score - gap_period

# Filter the datasets based on a fixed starting date: 5 years time window
gifts_score_start_period = cutoff_date_score - time_window

In [37]:
import pandas as pd

# Assuming 'drop_date', 'total_letters_sent', 'cost_unit' are columns in the 'campaigns' table
# Assuming 'campaignID', 'amount' are columns in the 'gifts' table

# Step 1: Compute features from the 'campaigns' table
cutoff_date_score = pd.to_datetime(cutoff_date_score)  

# Filter campaigns within the 5-year time window before cutoff date_score
campaigns_filtered = campaigns[(campaigns['date'] >= gifts_score_start_period) & (campaigns['date'] <= cutoff_date_score)]

# Compute total_letters_sent and total_cost
campaigns_aggregated = campaigns_filtered.groupby('campaignID').agg(
    drop_date=('date', 'first'),
    total_letters_sent=('lettersSent', 'sum')
).reset_index()


In [38]:
# Merge with gifts to get additional information
campaigns_with_gifts = pd.merge(
    campaigns_aggregated,
    gifts.groupby('campaignID').agg(
        count_donations_over30=('amount', lambda x: (x >= 30).sum()),
        sum_donations_over30=('amount', lambda x: x[x >= 30].sum()),
        avg_donations_over30=('amount', lambda x: x[x >= 30].mean()),
        avg_donation=('amount', 'mean'),
        min_donation=('amount', 'min'),
        max_donation=('amount', 'max'),
        total_earnings=('amount', 'sum')
    ).reset_index(),
    how='left',
    on='campaignID'
)

In [39]:
campaigns_susbset = campaigns[['campaignID', 'CostUnit']]

In [40]:
campaigns_with_gifts = pd.merge(campaigns_with_gifts, campaigns_susbset, on='campaignID', how='left')

In [41]:
campaigns_with_gifts['total_cost'] =campaigns_with_gifts['total_letters_sent'] * campaigns_with_gifts['CostUnit']

In [42]:
campaigns_with_gifts['Profit'] = campaigns_with_gifts['total_earnings'] - campaigns_with_gifts['total_cost']

In [43]:
campaigns_with_gifts['avg_response_rate_over30'] = campaigns_with_gifts['count_donations_over30'] / campaigns_with_gifts['total_letters_sent'] * 100

In [44]:
campaigns_with_gifts.describe()

,total_letters_sent,count_donations_over30,sum_donations_over30,avg_donations_over30,avg_donation,min_donation,max_donation,total_earnings,CostUnit,total_cost,Profit,avg_response_rate_over30
count,118.00,118.00,118.00,118.00,118.00,118.00,118.00,118.00,118.00,118.00,118.00,118.00
mean,20996.44,151.53,7348.66,48.39,32.65,6.13,255.54,8851.40,0.23,5290.52,3560.88,0.56
std,13085.60,212.41,11625.11,10.60,12.11,13.61,255.21,13694.96,0.16,6127.35,8926.94,0.50
min,5172.00,1.00,40.00,35.85,11.17,0.20,40.00,40.00,0.01,107.82,-7843.24,0.01
25%,7845.25,20.00,963.00,42.36,26.16,1.06,100.00,1193.45,0.11,1156.84,41.55,0.21
50%,21903.00,89.50,4535.55,46.16,29.84,2.25,200.00,5225.18,0.19,3467.31,999.14,0.42
75%,30902.25,205.00,9792.86,49.98,37.19,3.00,300.00,11397.86,0.31,7141.83,4566.64,0.81
max,69879.00,1676.00,95513.61,100.00,100.00,100.00,1430.61,98653.56,0.87,41927.40,68301.00,2.74


In [46]:
campaigns_with_gifts['avg_response_rate_over30'].describe()

count   118.00
mean      0.56
std       0.50
min       0.01
25%       0.21
50%       0.42
75%       0.81
max       2.74
Name: avg_response_rate_over30, dtype: float64

In [54]:
# Calculate global totals and averages
global_average_letters_sent = campaigns_with_gifts['total_letters_sent'].mean()
global_average_gift_value_over30 = campaigns_with_gifts['sum_donations_over30'].sum() / campaigns_with_gifts['count_donations_over30'].sum()
global_average_response_rate_over30 = campaigns_with_gifts['count_donations_over30'].sum() / campaigns_with_gifts['total_letters_sent'].sum() * 100
global_average_cost = campaigns_with_gifts['total_cost'].sum() / len(campaigns_with_gifts['total_cost'])
global_average_revenue = campaigns_with_gifts['avg_donation'].mean()
global_total_cost = campaigns_with_gifts['total_cost'].sum()
global_total_revenue_over30 = campaigns_with_gifts['sum_donations_over30'].sum() - campaigns_with_gifts['total_cost'].sum()
global_total_revenue = campaigns_with_gifts['total_earnings'].sum()
global_total_profit = campaigns_with_gifts['Profit'].sum()
global_average_profit = campaigns_with_gifts['Profit'].mean()
global_average_profit_over30 = campaigns_with_gifts['Profit'].mean()

# Create a summary DataFrame for global totals and averages
global_summary_df = pd.DataFrame({
    'global_avg_letters_sent' : global_average_letters_sent,
    'global_avg_gift_value_over30': global_average_gift_value_over30,
    'global_average_response_rate_over30': global_average_response_rate_over30,
    'global_average_cost': global_average_cost,
    'global_average_revenue': global_average_revenue,
    'global_total_cost': global_total_cost,
    'global_total_revenue': global_total_revenue,
    'global_total_revenue_over30': global_total_revenue_over30,
    'global_total_profit': global_total_profit,
    'global_average_profit': global_average_profit,
    'global_average_profit_over30': global_average_profit_over30
}, index=[0])  # Creating a DataFrame with a single row

# Export the global summary DataFrame to a CSV file
global_summary_df.to_csv('global_campaigns_summary.csv', index=False)

In [55]:
global_summary_df

,global_avg_letters_sent,global_avg_gift_value_over30,global_average_response_rate_over30,global_average_cost,global_average_revenue,global_total_cost,global_total_revenue,global_total_revenue_over30,global_total_profit,global_average_profit,global_average_profit_over30
0,20996.44,48.50,0.72,5290.52,32.65,624281.50,1044465.19,242859.91,420183.69,3560.88,3560.88


In [19]:
0.8*5304

4243.2

In [23]:
5304*0.025

132.6

In [28]:
132*48.5 

6402.0

In [17]:
# Export the result DataFrames to CSV files
global_summary_df.to_csv('campaigns_summary.csv', index=False)
campaigns_with_gifts.to_csv('campaigns_with_gifts.csv', index=False)